## Imports

In [34]:
import json
import pandas as pd
import numpy as np

## Lectura input

In [46]:
path = '/Users/herminiapadialromera/ironhackb/Proyecto_final/notebooks/input_.json'
pd.set_option('display.max_colwidth', None)
df_input_json = pd.read_json(path)
columnas_explode = ['Etapa', 'Formato', 'Temática', 'Objetivos'] 
df_input_json_merge = df_input_json.explode(['Etapa']).explode(['Formato']).explode(['Objetivos']).explode(['Temática'])
#df_input_json_merge

## Lectura del dataset de cursos 

In [47]:
data_course_df = pd.read_csv('/Users/herminiapadialromera/ironhackb/Proyecto_final/data/Datos_ampliados.csv', sep=";", encoding='utf8')
data_course_df = pd.DataFrame(data_course_df)
data_course_df['Tipo de contenido'].fillna('No innovador', inplace= True)
columnas_explode_curso = ['Etapa', 'Formato', 'Temática', 'Objetivos'] 
data_course_df = data_course_df.explode(['Etapa']).explode(['Formato']).explode(['Temática'])
#data_course_df


## Merge de los dos dataframe

In [48]:
df_merge = pd.merge(df_input_json_merge, data_course_df, on=['Provincia', 'Etapa', 'Formato', 'Temática'])
df_merge = df_merge.iloc[:,1:]
#df_merge

## FILTROS


   ## Filtro temática, etapa, formato


In [44]:
def filtro_tematica_etapa_formato (df, sesiones):
    tematica_unique_list = df_merge['Temática'].unique()
    formato_unique_list = df_merge['Formato'].unique()
    etapa_unique_list = df_merge['Etapa'].unique()

    filtro_tematica_df = pd.DataFrame()
    for tematica in tematica_unique_list:
        df_tematica_temp = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Temática'] == tematica)].iloc[:1,:]
        filtro_tematica_df = pd.concat([filtro_tematica_df, df_tematica_temp])
        #print("filtro_tematica_df ---> ", tematica)
        #display(filtro_tematica_df)

    filtro_formato_df = pd.DataFrame()
    for formato in formato_unique_list:
        df_formato = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Formato'] == formato)].iloc[:1,:]
        filtro_formato_df = pd.concat([filtro_formato_df, df_formato])
        #print("filtro_formato_df ---> ", formato)
        #display(filtro_formato_df)

    filtro_etapa_df = pd.DataFrame()
    for etapa in etapa_unique_list:
        df_etapa = df_merge[(df_merge['Nº Sesiones'] <= int(f"{sesiones}")) & (df_merge['Etapa'] == etapa)].iloc[:1,:]
        filtro_etapa_df = pd.concat([filtro_etapa_df, df_etapa])
        #print("filtro_etapa_df ---> ", etapa)
        #display(filtro_etapa_df)

    recomendaciones_df = pd.concat([filtro_tematica_df,filtro_formato_df,filtro_etapa_df]).drop_duplicates()
    return recomendaciones_df

In [49]:
tematica_formato_etapa_df = filtro_tematica_etapa_formato (df_merge, 5)
#tematica_formato_etapa_df

   ## Filtro precio

In [42]:
def filtro_precio (df):
    if (tematica_formato_etapa_df['Precios por taller'] == 'Menos de 100').any():
        precio_df = tematica_formato_etapa_df[(tematica_formato_etapa_df['Precio por taller'] < 100)]
        print("menos de 100 ---> existe")

    elif (tematica_formato_etapa_df['Precios por taller'] == 'Entre 100 y 300').any():
        precio_df = tematica_formato_etapa_df[(tematica_formato_etapa_df['Precio por taller'] >= 100) & (tematica_formato_etapa_df['Precio por taller'] <= 300)]
        print("entre 100 y 300 ---> existe")

    else:
        precio_df = tematica_formato_etapa_df[(tematica_formato_etapa_df['Precio por taller'] > 300)]
        print("más de 300 ---> existe")
    
    return precio_df

In [50]:
filtro_precio_df = filtro_precio (tematica_formato_etapa_df)
#filtro_precio_df

entre 100 y 300 ---> existe
